## MySQL Database 

**Author:** Whitney King
<br>
**Date:** 5/26/2018

Based on practice from Leetcode and other various examples from around the web using MySQL

In [19]:
import mysql.connector as mysql
import pandas as pd

def line():
    print('------------------------------------------------------')

def executeQuery(query):
    query = open(query, 'r').read()
    cur.execute(query)
    query_out = cur.fetchall()
    query_columns = pd.Series(cur.column_names)
    query_data = pd.DataFrame(query_out,
                              columns=query_columns)
    print(query)
    line()
    return(query_data)

def returnQuery(sqlFile):
    query = sqlFile
    query = executeQuery(query)
    return query

In [20]:
config = {
  'user': 'beta',
  'host': 'localhost',
  'database': 'leetcode',
  'raise_on_warnings': False,
  'use_pure': False
}

cnx = mysql.connect(**config)
cur = cnx.cursor()

### 2nd Highest Salary

Write a SQL query to get the second highest salary from the Employee table. If there is no second highest salary, then the query should return null.

In [3]:
#cur.execute('CREATE DATABASE leetcode')
#cnx.commit()
#cur.execute('USE leetcode')
#cur.execute('CREATE TABLE Employee (Id int, Salary int)')

In [4]:
#employee_table = 'INSERT INTO Employee (Id, Salary) VALUES (%s, %s)'
employee_data = (1, 100), (2, 200), (3, 300)

#cur.executemany(employee_table, employee_data)
#cnx.commit()

In [5]:
salary = returnQuery('queries/Leetcode/SecondHighestSalary.sql')
salary

SELECT MAX(Salary) AS SecondHighestSalary
FROM Employee
WHERE Salary < (SELECT MAX(Salary) FROM Employee)
------------------------------------------------------


,SecondHighestSalary
0,200


### Nth Highest Salary 
Write a SQL query to get the nth highest salary from the Employee table.

- Note: 3 can be replaced with any value of n, or this can be made into a function

In [6]:
n_salary = returnQuery('queries/Leetcode/NthHighestSalary.sql')
n_salary

SELECT MAX(Salary) AS NthHighestSalary
FROM Employee AS e1
WHERE 3 = (SELECT COUNT(DISTINCT (e2.Salary)) 
           FROM Employee AS e2
           WHERE e2.Salary >= e1.Salary)
------------------------------------------------------


,NthHighestSalary
0,100


### Non-Ordering Customers

Suppose that a website contains two tables, the Customers table and the Orders table. Write a SQL query to find all customers who never order anything.

In [7]:
#cur.execute('CREATE TABLE Customers (Id int, Name text)')
#cur.execute('CREATE TABLE Orders (Id int, CustomerId int)')

In [8]:
#customers_table = 'INSERT INTO Customers (Id, Name) VALUES (%s, %s)'
customers_data = (1, 'Joe'), (2, 'Henry'), (3, 'Sam'), (4, 'Max'), (5, 'James'), (6, 'James')

#orders_table = 'INSERT INTO Orders (Id, CustomerId) VALUES (%s, %s)'
orders_data = (1, 3), (2, 1), (3, 5)

#cur.executemany(customers_table, customers_data)
#cur.executemany(orders_table, orders_data)
#cnx.commit()

In [9]:
non_cust = returnQuery('queries/Leetcode/NonOrderingCustomers.sql')
non_cust

SELECT Name AS Customers
FROM Customers AS c1
WHERE c1.Id NOT IN (SELECT c2.Id
                    FROM Customers AS c2
                        JOIN Orders AS o 
                            ON o.CustomerId = c2.Id)

------------------------------------------------------


,Customers
0,Henry
1,Max
2,James


### Friend Request Acceptance Rate

Write a query to find the overall acceptance rate of requests rounded to 2 decimals, which is the number of acceptance divide the number of requests.

- The accepted requests are not necessarily from the table friend_request. 
    - In this case, you just need to simply count the total accepted requests (no matter whether they are in the original requests), and divide it by the number of requests to get the acceptance rate.
- It is possible that a sender sends multiple requests to the same receiver, and a request could be accepted more than once. In this case, the ‘duplicated’ requests or acceptances are only counted once.
- If there is no requests at all, you should return 0.00 as the accept_rate.

In [3]:
#cur.execute('CREATE TABLE friend_request (sender_id int, send_to_id int, request_date date)')
#cur.execute('CREATE TABLE request_accepted (requester_id int, accepter_id int, accept_date date)')

In [4]:
#friends_table = 'INSERT INTO friend_request (sender_id, send_to_id, request_date) VALUES (%s, %s, %s)'
friends_data = [
           (1, 2, '2016_06-01'), (1, 3, '2016_06-01')
         , (1, 4, '2016_06-01'), (2, 3, '2016_06-02')
         , (3, 4, '2016_06-09')#, (4, 5, '2016_06-09')
          ]

#requests_table = 'INSERT INTO request_accepted (requester_id, accepter_id, accept_date) VALUES (%s, %s, %s)'
requests_data = [
           (1, 2, '2016_06-03'), (1, 3, '2016_06-08')
         , (2, 3, '2016_06-08'), (3, 4, '2016_06-09')
         #, (3, 4, '2016_06-10')
           ]

#cur.executemany(friends_table, friends_data)
#cur.executemany(requests_table, requests_data)
#cnx.commit()

In [6]:
friend_reqs = returnQuery('queries/Leetcode/FriendRequests1.sql')
friend_reqs

SELECT 
ROUND(
  IFNULL(
    (
    (SELECT 1.0 * COUNT(*)
     FROM (SELECT DISTINCT requester_id, accepter_id
           FROM request_accepted) AS a) -- Accepted
     
    /  -- COUNT(Accpted) / COUNT(Sent)
    
    (SELECT COUNT(*)
     FROM (SELECT DISTINCT sender_id, send_to_id
           FROM friend_request) AS s) -- Sent
    ), 
  0), 
2) AS accept_rate
------------------------------------------------------


,accept_rate
0,0.80


**Can you write a query to return the accept rate but for every month?**

In [7]:
friend_reqs_month = returnQuery('queries/Leetcode/FriendRequests2.sql')
friend_reqs_month

SELECT a.month AS month
	, IF(r.requested = 0, 0.00, 
	  ROUND(a.accepted / r.requested, 2)) AS accept_rate
FROM
	(SELECT 1.0 * COUNT(DISTINCT requester_id, accepter_id) AS accepted,
	 MONTH(accept_date) AS month 
	 FROM request_accepted) AS a
   ,
	(SELECT COUNT(DISTINCT sender_id, send_to_id) AS requested,
	 MONTH(request_date) AS month 
	 FROM friend_request) AS r

WHERE a.month = r.month
GROUP BY a.month

------------------------------------------------------


,month,accept_rate
0,6,0.80


**How about the cumulative accept rate for every day?**

This should be a count of acceptance rate for number of requests accepted on a given day divided by numbers of requests sent on a given day, regardless of who sent or accepted them.

In [11]:
friend_reqs_day = returnQuery('queries/Leetcode/FriendRequests3.sql')

SELECT date_table.dates AS date,
       IFNULL(ROUND(
	COUNT(DISTINCT requester_id, accepter_id) / 
	COUNT(DISTINCT sender_id, send_to_id)
     , 2), 0.00) AS accept_rate
     
FROM request_accepted AS a, friend_request AS r,
(
  SELECT request_date AS dates 
  FROM friend_request
	UNION
  SELECT accept_date 
  FROM request_accepted
	ORDER BY dates
) AS date_table

WHERE a.accept_date <= date_table.dates
AND r.request_date <= date_table.dates
GROUP BY date_table.dates
------------------------------------------------------


In [9]:
friend_reqs_day

,date,accept_rate
0,2016-06-03,0.25
1,2016-06-08,0.75
2,2016-06-09,0.80
3,2016-06-10,0.80


### Who Has the Most Friends?

- It is guaranteed there is only 1 people having the most friends.
- The friend request could only been accepted once, which mean there is no multiple records with the same requester_id and accepter_id

In [21]:
most_friends = returnQuery('queries/Leetcode/MostFriends1.sql')
most_friends

SELECT num.friends AS id,
		 COUNT(1) AS num
	FROM
	(
	 SELECT requester_id AS friends 
	 FROM request_accepted
		UNION ALL 
	 SELECT accepter_id
	 FROM request_accepted
	) AS num
	 
GROUP BY num.friends
ORDER BY num DESC
LIMIT 1
------------------------------------------------------


,id,num
0,2,3


**In the real world, multiple people could have the same most number of friends, can you find all these people in this case?**

In [22]:
most_friends2 = returnQuery('queries/Leetcode/MostFriends2.sql')
most_friends2

SELECT friends.id AS id,
	COUNT(1) AS num
FROM
(
SELECT requester_id AS id 
FROM request_accepted
	UNION ALL 
SELECT accepter_id
FROM request_accepted
) AS friends
			 
GROUP BY id
HAVING num = (
	SELECT MAX(num) AS max
	FROM
	(
	SELECT friends.id AS id, COUNT(1) AS num
		FROM
		(
		SELECT requester_id AS id 
		FROM request_accepted
			UNION ALL 
		SELECT accepter_id
		FROM request_accepted
		) AS friends
        
	GROUP BY id
	) AS max
)
ORDER BY id DESC

------------------------------------------------------


,id,num
0,3,3
1,2,3


### Department Top Three Salaries

Write a SQL query to find employees who earn the top three salaries in each of the department.

In [3]:
#cur.execute('DROP TABLE Employee')
#cnx.commit()
#cur.execute('CREATE TABLE Employee (Id int, Name text, Salary int, DepartmentId int)')
#cur.execute('CREATE TABLE Department (Id int, Name text)')

In [4]:
#employees_table = 'INSERT INTO Employee (Id, Name, Salary, DepartmentId) VALUES (%s, %s, %s, %s)'
employees_data = [
                    (1, 'Joe', 70000, 1), (2, 'Henry', 80000, 2)
                  , (3, 'Sam', 60000, 2), (4, 'Max', 90000, 1)
                  , (5, 'Janet', 69000, 1), (6, 'Randy', 85000, 1)
                 ]

#departments_table = 'INSERT INTO Department (Id, Name) VALUES (%s, %s)'
departments_data = (1, 'IT'), (2, 'Sales')

In [ ]:
#cur.executemany(employees_table, employees_data)
#cur.executemany(departments_table, departments_data)
#cnx.commit()

To begin, we will look at what it takes to get a view of the employees with the highest salaries in each department.

In [39]:
dept_sal = returnQuery('queries/Leetcode/DepartmentSalaries.sql')
dept_sal

SELECT d.Name AS Department
	 , e.Name AS Employee
	 , Salary
FROM Employee AS e
	JOIN Department AS d 
		ON e.DepartmentId = d.Id
        
WHERE (e.DepartmentId, Salary) IN (
	SELECT DepartmentId, MAX(Salary)
	FROM Employee AS e
	GROUP BY DepartmentId
)
ORDER BY Salary DESC
------------------------------------------------------


,Department,Employee,Salary
0,IT,Max,90000
1,Sales,Henry,80000


Next, we take this logic, and merge it with the Nth highest salary logic we did earlier to get the query and results we need

In [42]:
dept_sal2 = returnQuery('queries/Leetcode/DepartmentSalaries2.sql')
dept_sal2

SELECT d.Name AS Department
	 , e1.Name AS Employee
	 , Salary
FROM Employee AS e1
	JOIN Department AS d 
		ON e1.DepartmentId = d.Id
        
WHERE 3 > (
	SELECT COUNT(DISTINCT e2.Salary)
	FROM Employee AS e2
	WHERE e2.Salary > e1.Salary
	AND e1.DepartmentId = e2.DepartmentId
)
ORDER BY Department, Salary DESC
------------------------------------------------------


,Department,Employee,Salary
0,IT,Max,90000
1,IT,Randy,85000
2,IT,Joe,70000
3,Sales,Henry,80000
4,Sales,Sam,60000
